In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt



In [3]:
words = open('names.txt', 'r').read().splitlines()

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

print(itos)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
block_size = 3  # context length: how many characters do we take to predict the next one?
X, Y = [], []

for w in words[:5]:
    print(w)
    context = [0] * block_size  # initialize with all dots
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '----->', itos[ix])
        # slide the window, by popping and appending
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)


emma
... -----> e
..e -----> m
.em -----> m
emm -----> a
mma -----> .
olivia
... -----> o
..o -----> l
.ol -----> i
oli -----> v
liv -----> i
ivi -----> a
via -----> .
ava
... -----> a
..a -----> v
.av -----> a
ava -----> .
isabella
... -----> i
..i -----> s
.is -----> a
isa -----> b
sab -----> e
abe -----> l
bel -----> l
ell -----> a
lla -----> .
sophia
... -----> s
..s -----> o
.so -----> p
sop -----> h
oph -----> i
phi -----> a
hia -----> .


In [21]:
Y

tensor([ 5, 13, 13,  1,  0, 15, 12,  9, 22,  9,  1,  0,  1, 22,  1,  0,  9, 19,
         1,  2,  5, 12, 12,  1,  0, 19, 15, 16,  8,  9,  1,  0])

In [9]:
C = torch.randn((27, 2))

C

tensor([[-1.3458, -0.3413],
        [-1.9735,  1.0949],
        [-0.6049, -0.3334],
        [ 1.0611,  0.3853],
        [-0.3184,  0.1318],
        [-0.3362, -0.8939],
        [ 1.2128, -0.0724],
        [ 0.1820,  0.4361],
        [ 0.0064,  1.2892],
        [ 0.7103,  0.9900],
        [-0.4593,  0.3496],
        [ 0.5218,  0.6501],
        [ 0.2293, -1.2397],
        [ 1.4145,  0.9512],
        [-0.4621,  0.0926],
        [-1.0747,  0.2503],
        [-1.5099, -0.1681],
        [ 0.0030, -0.9490],
        [ 0.5008, -0.6192],
        [ 1.1905,  0.6301],
        [-1.1123, -0.3211],
        [-0.4604,  1.3740],
        [-2.3995,  0.3684],
        [-0.8702, -0.2539],
        [-1.1146, -0.1335],
        [-0.4274,  0.7138],
        [ 0.8277,  0.7871]])

In [19]:
emb = C[X]
W1 = torch.randn((block_size * 2, 100))
b1 = torch.randn(100)

#torch.cat(torch.unbind(emb, dim=1), dim=1)

h = torch.tanh(emb.view(-1, block_size * 2) @ W1 + b1)


In [20]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

logits = h @ W2 + b2
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)

prob


tensor([[5.7290e-10, 5.9980e-10, 7.0522e-09, 4.2503e-08, 3.0369e-12, 3.9557e-05,
         7.9705e-05, 3.5665e-11, 9.9749e-01, 4.5216e-11, 1.3408e-11, 5.0751e-13,
         1.7765e-03, 9.1036e-13, 1.7516e-06, 6.2864e-10, 4.4767e-11, 1.4439e-13,
         1.7971e-13, 1.4110e-10, 2.7971e-10, 2.2152e-05, 5.9424e-04, 3.7384e-11,
         7.6717e-19, 7.9684e-14, 2.6957e-09],
        [2.6854e-11, 1.0224e-07, 1.6884e-07, 3.8017e-09, 6.4457e-10, 5.9010e-08,
         1.6954e-04, 2.9780e-13, 8.2517e-01, 1.8016e-08, 6.2891e-12, 9.8109e-13,
         1.0667e-04, 2.3664e-12, 4.6134e-08, 8.8354e-10, 8.4909e-12, 8.5858e-16,
         2.1753e-12, 1.2359e-10, 7.7298e-14, 3.3554e-07, 1.7456e-01, 1.0999e-10,
         1.3407e-15, 7.7832e-13, 2.0590e-11],
        [5.4152e-10, 2.5184e-04, 3.4475e-08, 6.5119e-04, 1.4228e-08, 9.0266e-08,
         1.0170e-07, 7.5725e-11, 5.0494e-04, 2.2565e-06, 5.7964e-13, 7.2055e-10,
         4.9046e-09, 2.1254e-04, 1.1931e-12, 7.5540e-09, 6.5248e-05, 8.1858e-14,
         8.7737e-

In [ ]:
loss = -prob[torch.arange(32), Y].log().mean()

tensor([3.9557e-05, 2.3664e-12, 2.1254e-04, 9.7035e-10, 2.8368e-05, 6.2864e-10,
        7.6228e-05, 1.4095e-11, 1.5443e-05, 1.0634e-07, 6.6539e-07, 2.6071e-15,
        5.9980e-10, 1.7486e-06, 4.9489e-09, 1.0810e-06, 4.5216e-11, 2.7171e-07,
        2.0904e-10, 2.9195e-09, 1.9563e-06, 8.5220e-06, 2.5328e-06, 2.3565e-05,
        9.3106e-14, 1.4110e-10, 3.8462e-08, 5.0490e-07, 1.2418e-04, 1.0091e-11,
        3.0136e-13, 1.1045e-05])